In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
os.chdir("/work/users/minhnth/projects/GIP/data/HLA")

In [4]:
data = pd.read_csv("HLA1_chr6.0.2.missing.hap", header = None, sep = " ")

/tmp/ipykernel_75336/1081465367.py:1: DtypeWarning: Columns (1810,1811) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("HLA1_chr6.0.2.missing.hap", header = None, sep = " ")


In [5]:
data

,0,1,2,3,4,5,6,7,8,9,...,4998,4999,5000,5001,5002,5003,5004,5005,5006,5007
0,0,0,0,0,0,0,0,0,?,?,...,0,0,?,?,0,0,?,?,0,0
1,0,0,0,0,0,0,0,0,?,?,...,0,0,0,1,0,0,0,0,0,0
2,?,?,?,?,0,0,0,0,0,0,...,0,0,?,?,?,?,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,?,?,0,0,0,0,0,0,?,?
4,?,?,0,0,?,?,?,?,?,?,...,?,?,0,0,0,0,0,0,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1054,?,?,0,0,?,?,?,?,0,0,...,0,0,0,0,0,0,?,?,0,0
1055,?,?,0,0,?,?,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1056,0,0,0,0,0,0,?,?,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
a = data.to_numpy()

In [9]:
a.astype(float)

ValueError: could not convert string to float: '?'

In [10]:
b = a == "?"

In [11]:
b

array([[False, False, False, ...,  True, False, False],
       [False, False, False, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       ...,
       [ True,  True, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [ True,  True, False, ..., False,  True,  True]])

In [12]:
a[b] = np.nan

In [14]:
a.astype(float)

array([[ 0.,  0.,  0., ..., nan,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [nan, nan, nan, ...,  0.,  0.,  0.],
       ...,
       [nan, nan,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [nan, nan,  0., ...,  0., nan, nan]])

In [ ]:
a